# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Контрольная работа

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

Задание 1.1

In [10]:
import pandas as pd
from nltk.tokenize import word_tokenize 

df = pd.read_csv("preprocessed_descriptions.csv") 
print(df.head())
words = ' '.join([str(i) for i in df['description']])
unique_words = set(word_tokenize(words))
print(unique_words)



                                       name     id  minutes  contributor_id  \
0     george s at the cove  black bean soup  44123       90           35193   
1        healthy for them  yogurt popsicles  67664       10           91970   
2              i can t believe it s spinach  38798       30            1533   
3                      italian  gut busters  35173       45           22724   
4  love is in the air  beef fondue   sauces  84797       25            4470   

    submitted  n_steps                                        description  \
0  2002-10-25      NaN  an original recipe created by chef scott meska...   
1  2003-07-26      NaN  my children and their friends ask for my homem...   
2  2002-08-29      NaN            these were so go, it surprised even me.   
3  2002-07-27      NaN  my sister-in-law made these for us at a family...   
4  2004-02-23      4.0  i think a fondue is a very romantic casual din...   

   n_ingredients                           preprocessed_descri

Задание 1.2

In [4]:
from nltk import edit_distance
import random 
pairs = [(word1, word2) for word1, word2 in zip(list(unique_words), list(unique_words)[1:])]

random_pairs = random.sample(pairs, 5) 

print(random_pairs)

distances = []
for pair in random_pairs:
    distance = edit_distance(pair[0], pair[1])
    distances.append(distance)
print("Расстояния редактирования для 5 случайно выбранных пар слов:")
print(distances)

[('skye', 'etowah'), ('semi-frozen', 'mined'), ('twang', 'baking'), ('bowties', '504523'), ('after-dinner', 'leader')]
Расстояния редактирования для 5 случайно выбранных пар слов:
[6, 8, 4, 7, 8]


Задание 1.3

In [5]:
def get_closest_words(word, k):
    all_words = list(words)

    sorted_words = sorted(all_words, key=lambda x: edit_distance(x, word))

    return sorted_words[:k]


Задание 2.1

In [8]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

df = pd.read_csv('preprocessed_descriptions.csv')
words = set(word_tokenize(" ".join([str(i) for i in df["description"]])))

data = []
for word in words:
    stemmed_word = SnowballStemmer("english").stem(word)
    normalized_word = WordNetLemmatizer().lemmatize(word)
    data.append([word, stemmed_word, normalized_word])

df_result = pd.DataFrame(data, columns=['word', 'stemmed_word', 'normalized_word'])
df_result.set_index('word', inplace=True)
print(df_result)

                stemmed_word normalized_word
word                                        
sugar/ketchup  sugar/ketchup   sugar/ketchup
lugs                     lug             lug
milk/water          milk/wat      milk/water
eremich              eremich         eremich
countryside       countrysid     countryside
...                      ...             ...
notation               notat        notation
high                    high            high
heighten            heighten        heighten
tommie                 tommi          tommie
listed                  list          listed

[29768 rows x 2 columns]


Задание 2.2

In [10]:
stop_words_count = 0
total_words_count = len(words)
words = list(words)
for word in stopwords.words("english"):
    stop_words_count += words.count(word)

stop_word_ratio = stop_words_count / total_words_count
print("Доля стоп-слов:", stop_word_ratio * 100, "%")

top_10_before = df.groupby('description').size().sort_values(ascending=False).head(10)
top_10_after = df_result.groupby(level=0).size().sort_values(ascending=False).head(10)

print("Топ-10 самых часто употребляемых слов до удаления стоп-слов:")
print(top_10_before.index)

print()
print("Топ-10 самых часто употребляемых слов после удаления стоп-слов:")
print(top_10_after.index)

Доля стоп-слов: 0.49381886589626445 %
Топ-10 самых часто употребляемых слов до удаления стоп-слов:
Index(['yum', 'from southern living.', 'southern living',
       'from taste of home.', 'yum!', 'from cooking light.', 'rachael ray',
       'yummy', 'cooking light.', 'cooking light'],
      dtype='object', name='description')

Топ-10 самых часто употребляемых слов после удаления стоп-слов:
Index(['!', 'palm-yay', 'pampero', 'pampered', 'pamesean', 'pamela', 'pam',
       'palomilla', 'palomara', 'palms'],
      dtype='object', name='word')


Задание 3.1

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('preprocessed_descriptions.csv')
recipes = df.sample(n=5)
recipe_descriptions = []
for recipe in recipes['description']:
    recipe_descriptions.append(recipe)

vectorizer = TfidfVectorizer()
recipe_vectors = vectorizer.fit_transform(recipe_descriptions)
print(recipe_vectors.toarray())

[[0.         0.28122937 0.         0.         0.         0.
  0.         0.         0.         0.28122937 0.         0.
  0.         0.         0.         0.         0.         0.
  0.28122937 0.         0.         0.         0.28122937 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.18834258
  0.         0.37668517 0.22689408 0.         0.         0.28122937
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.22689408 0.         0.         0.28122937 0.28122937 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.28122937
  0.         0.         0.22689408 0.18834258 0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.14863669 0.22194132 0.11097066 0.
  0.   

Заданние 3.2